In [20]:
import glob
import pandas as pd
import numpy as np
import os, sys
import collections
import math
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [21]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier 

In [22]:
from numba.typed import List
from numba import jit, njit, vectorize

In [23]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [24]:
sample_rate = 50  # number of observation per second based on dataset documentation(150 samples in 3 second)

sliding_size = int((1/3) * sample_rate)  # (50/150) i.e 100 out of 150 are overlapping (number of skipped datapoints to start next window)
print(sliding_size)

16


# Feature sets

In [25]:
@njit()
def mean_crossing_rate(col):
    # col = np.array(values)
    normalized = col - col.mean()  # to make elements of array possitive or negetive
    return ((normalized[:-1] * col[1:]) < 0).sum()  # Zero-Crossing_rate

@njit()
def iqr(window):  # inter-quartile range
    Q1 = np.median(window[:len(window)//2])  # First quartile (Q1) 
    Q3 = np.median(window[len(window)//2:])  # Third quartile (Q3) 
    IQR = Q3 - Q1 # Interquartile range (IQR) 
    return(IQR) 
@njit()
def calc_sma_for_window(data):
    return np.sum(data) / len(data)  
@njit()
def get_min(x):
    m = np.min(x)
    return m
@njit()
def get_max(x):
    m = np.max(x)
    return m
@njit()
def get_mean(x):
    m = np.mean(x)
    return m
@njit()
def get_var(x):
    m = np.var(x)
    return m
@njit()
def get_sum(x):
    m = x.sum()
    return m 
@njit()
def get_median(x):
    m = np.median(x)
    return m 
@njit()
def get_std(x):
    m = np.median(x)
    return m 
@njit()
def get_rng(x):
    n = np.max(x)
    m = np.min(x)
    z = n-m
    return z 

def get_entropy(Y):
    """
    Also known as Shanon Entropy
    Reference: https://en.wikipedia.org/wiki/Entropy_(information_theory)
    """
    unique, count = np.unique(Y, return_counts=True, axis=0)
    prob = count/len(Y)
    en = np.sum((-1)*prob*np.log2(prob))
    return en

In [26]:
def Energy(frame):
    return sum( [ abs(x)**2 for x in frame ] ) / len(frame)

# Feature extraction and Vector_Normalization_and_mean_removal on signal

In [27]:
def Features(window):# mean, std,max,min and zero-crossing-rate
    wind = window.iloc[:, :-1]
    win = np.array(wind)
    
    data_normalized = preprocessing.normalize(win, norm  = 'l1') # normalization on signal
    win = data_normalized
    data_standardized = preprocessing.scale(win) # mean removal on signal
    win = data_standardized
    features = []
    
    features = []
    
    features.append(get_mean(win))
    features.append(get_median(win))
    features.append(get_std(win))
    features.append(get_min(win))
    features.append(get_max(win))
    features.append(get_sum(win))
    features.append(calc_sma_for_window(win))
    features.append(get_rng(win))
    features.append(get_entropy(win))
    mean_crossing = [mean_crossing_rate(win[:, i]) for i in range(win.shape[1])]
    features.append(np.array(mean_crossing))
    IQR = iqr(win)
    features.append(np.array(IQR))
    energy_measure = Energy(win)
    features.append(np.array(energy_measure))
 

    
    features = np.hstack(features).tolist()
    
    label = window.iloc[:, -1].mode()[0]  ## select the most frequent label as the label of the window
    features.append(label)
    return features

In [28]:
def windowing_dataset(dataset, win_size, feature_extraction_function, subject_id, overlap=False):
    windowed_dataset = []
    win_count = 0
    if overlap:
        step_size = sliding_size  # for Overlapping technique
    else:
        step_size = win_size  # for Non-overlapping technique

    for index in range(0, dataset.shape[0], step_size):
        start = index
        end = start + win_size
        # to assure all of windows are equal in size
        if (end <= dataset.shape[0]):
            window = dataset.iloc[start:end, :].reset_index(drop=True)
            win_count = win_count + 1
            features = feature_extraction_function(window)

            windowed_dataset.append(features)

    final = pd.DataFrame(windowed_dataset)
    final.insert(0, 'group', subject_id)  # to use in Subject CV
    return final

In [29]:
def Preprocessing(dataset_path, overlapping):
    feature_function = Features
    win_size = 3

    print("Start for win size {}".format(win_size))
    datapoints_per_window = int(win_size * sample_rate)

    print(feature_function.__name__)

    ALL = []
   
    for subject in range(1,18):
        file_path = dataset_path + '\subject{0}_ideal.csv'.format(subject)
        all_cols = []
        
        
#########################################################################################################################
        for i in range(2, 117, 13):# indices of accelarations
            indices = list(range(i, i + 13))
            all_cols.extend(indices)

        all_cols.append(119)  # label index

        tmp_db = pd.read_csv(file_path, header=None, usecols=all_cols, sep='\t')
        tmp_db.columns = list(range(tmp_db.shape[1]))  # re-index the columns

        transformed_db = windowing_dataset(tmp_db, datapoints_per_window, feature_function, subject,
                                                   overlap=overlapping)

        ALL.append(transformed_db)
#########################################################################################################################
       
    #final_dataset = pd.DataFrame()
    ALL_dataset = pd.DataFrame()
    ALL_dataset = ALL_dataset.append(ALL, ignore_index=True)
    
    return ALL_dataset
   

In [30]:
def subject_cross_validation(X, Y, groups, classifier):
    f1 = []
    logo = LeaveOneGroupOut()
    i = 0

    for train_index, test_index in logo.split(X, Y, groups=groups):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        i += 1
        classifier.fit(X_train, y_train)

        y_pred = classifier.predict(X_test)
        f = f1_score(y_true=y_test, y_pred=y_pred, average='micro')
        
        print('Model-',i ,' -',' f1 score: ', f)
       
        f1.append(f)
    return np.mean(f1)

In [31]:
def classifier(dataset, model):
    results = dict()
    win_size = float(3)
    print('window_size = ', win_size,' sec')

    dataset = dataset
    groups = dataset['group']
    X = dataset.iloc[:, 1:-1]
    X = np.array(X)
        
    Y = dataset.iloc[:, -1]
    Y = np.array(Y)

    for model_name, mod in model.items():
        f1 = 0

        f1 = subject_cross_validation(X, Y, groups, mod)

        if win_size in results:
            results[win_size].append(f1)
        else:
            results[win_size] = [f1]


        results = collections.OrderedDict(sorted(results.items()))

        final = []
        col = list(model.keys())
        col.insert(0, "window-size")
        final.append(col)
        for k, v in results.items():
            tmp = []
            tmp.append([k])
            tmp.append(v)
            flattened = [val for sublist in tmp for val in sublist]
            final.append(flattened)

    accuracy = final[1][1]
    print(accuracy)
    return accuracy

In [32]:
mod = {'RF': RandomForestClassifier(n_estimators=40, random_state=42, n_jobs=-1)}

In [33]:
dataset_csv_path = r"D:\projec\proj\data"

overlapping = 1  # input 0 for non overlapping, 1 for overlapping

ALL = Preprocessing(dataset_path=dataset_csv_path, overlapping=bool(int(overlapping)))

Start for win size 3
Features


In [34]:
ALL.head(10)

,group,0,1,2,3,4,5,6,7,8,...,235,236,237,238,239,240,241,242,243,244
0,1,-1.781260e-16,0.008714,0.008714,-3.827438,6.294121,-3.126110e-12,-2.084074e-14,10.121559,7.228819,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
1,1,4.474578e-17,0.004306,0.004306,-4.106611,5.924324,7.852885e-13,5.235257e-15,10.030935,7.228819,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
2,1,-6.393113e-17,0.001779,0.001779,-3.775134,5.342686,-1.121991e-12,-7.479943e-15,9.117820,7.228819,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
3,1,4.967220e-17,-0.000963,-0.000963,-5.630843,5.997940,8.717471e-13,5.811647e-15,11.628783,7.228819,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
4,1,1.827977e-16,0.030539,0.030539,-4.375715,5.325119,3.208100e-12,2.138734e-14,9.700834,7.228819,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
5,1,1.129328e-16,0.034320,0.034320,-4.369618,5.241926,1.981970e-12,1.321313e-14,9.611543,7.228819,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
6,1,1.635286e-17,0.030861,0.030861,-4.774677,6.098896,2.869927e-13,1.913284e-15,10.873574,7.228819,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
7,1,-1.561271e-17,0.039880,0.039880,-5.192394,6.947431,-2.740030e-13,-1.826687e-15,12.139825,7.228819,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
8,1,-3.782982e-18,0.020500,0.020500,-4.216488,6.995583,-6.639134e-14,-4.426089e-16,11.212071,7.228819,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
9,1,4.736952e-17,-0.001448,-0.001448,-4.479881,6.919605,8.313350e-13,5.542233e-15,11.399486,7.228819,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1


In [35]:
print('ALL_accuracy = ', )
ALL_accuracy = classifier(dataset=ALL, model=mod)
ALL_accuracy 

ALL_accuracy = 
window_size =  3.0  sec
Model- 1  -  f1 score:  0.9136363636363637
Model- 2  -  f1 score:  0.922805780754318
Model- 3  -  f1 score:  0.9328
Model- 4  -  f1 score:  0.9173932996787517
Model- 5  -  f1 score:  0.8379310344827586
Model- 6  -  f1 score:  0.88
Model- 7  -  f1 score:  0.96398891966759
Model- 8  -  f1 score:  0.8867127496159755
Model- 9  -  f1 score:  0.8683760683760683
Model- 10  -  f1 score:  0.9532263571184705
Model- 11  -  f1 score:  0.924375
Model- 12  -  f1 score:  0.9016959798994975
Model- 13  -  f1 score:  0.8394016370307648
Model- 14  -  f1 score:  0.9053150299190426
Model- 15  -  f1 score:  0.9224772628843654
Model- 16  -  f1 score:  0.8915266617969321
Model- 17  -  f1 score:  0.8596424496006086
0.9012532114389122


0.9012532114389122

In [36]:
d ={'ALL': [ALL_accuracy]}

In [37]:
df_accuracy = pd.DataFrame(data=d)

In [38]:
df_accuracy

,ALL
0,0.901253
